In [27]:
####################################################################################################################
### DE & particle velocity prediction ###
####################################################################################################################
### This script shows the pyhon code (jupyter notebook) used for the ML prediction models described in the manuscript: 
### "Application of machine learning for the prediction of particle velocity distribution and deposition efficiency for cold spraying titanium powder".

# Import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
import h5py
import joblib
import math

# Load the models
model1_NN = load_model("model1_NN_vel_pred_20240228.h5")
model2_NN = load_model("model2_NN_count_pred_20240228.h5")
scalerDE = joblib.load("standard_scaler_vel_DE_240228.joblib")

# User input and prediction output
try:
    powder_feeders_speed_1_2_rpm = float(input("Enter powder feeder rate [RPM]: "))
    print("You entered:", powder_feeders_speed_1_2_rpm)
    if not 1 <= powder_feeders_speed_1_2_rpm <= 20:
        raise ValueError("Powder feeder rate must be between 1 and 20 RPM.")
    Gas_temperature_C = float(input("Enter Gas temperature [°C]: "))
    if not 750 <= Gas_temperature_C <= 1100:
        raise ValueError("Gas temperature must be between 750 and 1100°C.")
    print("You entered:", Gas_temperature_C)
    print("---------------------------------------------------------------------")

    # Transform RPM into g/min
    Test_PFR_de_data = 0.2934 * powder_feeders_speed_1_2_rpm - 0.1538 

    # Calculation of window of deposition
    Test_Temp_de_data=Gas_temperature_C

    F1=1
    F2=0.8
    d_p = 32
    T_0_celsius = Test_Temp_de_data
    T_0 = T_0_celsius + 273.15
    P_0 = 49
    p_p = 4540
    L_ex = (160 -22.6 + 70)/1000
    ER = 4.9
    sigma = 240 * 10**6
    T_pm = 1941
    c_p = 540

    # Calculate latitudes
    lat_min = -5.0
    lat_max = 5.0
    step = 0.25
    latitudes = []
    current_lat = lat_min
    while current_lat <= lat_max:
        latitudes.append(current_lat)
        current_lat += step

    # Impact temperature
    T_pi = 75.4 + 5.368*math.log(d_p) + 0.205*T_0 +0.161*T_0*math.log(d_p) - 0.071*P_0 - 0.0057*p_p - 307.7*L_ex - 2.04*ER
    # Critical velocity   
    value = (4*F1*sigma*(1-((T_pi-293)/(T_pm-293)))/p_p)+F2*c_p*(T_pm-T_pi)
    critical_vel= math.sqrt(value)
    # Erosion velocity
    erosion_vel = 2*critical_vel

    def round_down_to_0_25(value):
        return round(value * 4) / 4

    data_for_F_evaluation = {
    'Lat position': latitudes,
    'Temperature_degreeC': [Test_Temp_de_data]*len(latitudes),
    'powder_feed_rate_g_per_hr': [Test_PFR_de_data]*len(latitudes)
    }
    df_data_for_F_evaluation = pd.DataFrame(data_for_F_evaluation)
    x2 = df_data_for_F_evaluation['Lat position']

    #Model velocity and particle occurance predictions
    data_x_scaled = scalerDE.transform(df_data_for_F_evaluation)
    test_predict1 = model1_NN.predict(data_x_scaled, verbose=0)
    test_predict2 = model2_NN.predict(data_x_scaled, verbose=0)
    y1_2 = test_predict1
    y1_2_list =y1_2.tolist()
    flattened_list = [item for sublist in y1_2_list for item in sublist]
    y1_2_list = flattened_list
    y2_2 = test_predict2
    y2_2_list =y2_2.tolist()
    flattened_list = [item for sublist in y2_2_list for item in sublist]
    y2_2_list = flattened_list

    # Determination of DE
    sum_all_particles = sum(y2_2_list)
    if sum_all_particles != 0:
        y2_2_list_percent = [value / sum_all_particles for value in y2_2_list]
    else:
        y2_2_list_percent = [0 for value in y2_2_list]
    positions_vel = [index for index, value in enumerate(y1_2_list) if value > critical_vel and value < erosion_vel]
    sum_particle_adhere = sum([y2_2_list_percent[index] for index in positions_vel])
    DE = sum_particle_adhere*100
    print(f"Predicted DE:{round(DE,2)}%")

    #Determination of Avg velocity
    rounded_list2 = [round_down_to_0_25(value) for value in x2]
    x2 = rounded_list2
    multiplied_list2_2 = [value * 100 for value in y2_2_list_percent] 
    y2_2_list_percent=multiplied_list2_2
    result_list = [x * y/100 for x, y in zip(y1_2, y2_2_list_percent)]
    list_sum = sum(result_list)
    Avg_vel=list_sum
    avg_vel=Avg_vel[0]
    print(f"Overall average particle velocity: {avg_vel:.2f} m/s")
    print("---------------------------------------------------------------------")

    # Create figure
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman']
    plt.rcParams['font.size'] = 30

    plt.figure(figsize=(12, 8))
    ax1 = plt.gca()
    ax1.plot(x2, y1_2, 'r-', label='predicted particle velocity')
    ax1.set_xlabel('Y position', fontdict={'fontsize': 30, 'fontname': 'Times New Roman'})
    ax1.set_ylabel('Velocity', color='black', fontdict={'fontsize': 30, 'fontname': 'Times New Roman'})
    ax1.set_ylim(top=2000)
    ax1.axhline(y=critical_vel, color='red', linestyle='--', label='critical velocity')
    ax1.axhline(y=erosion_vel, color='red', linestyle='--', label='erosion velocity')
    ax1.legend(loc='upper left', fontsize=30, bbox_to_anchor=(1.2, 0.8), borderaxespad=0., frameon=False)

    ax2 = ax1.twinx()
    bar_width = 0.2
    ax2.bar(x2, y2_2_list_percent, color='green', alpha=0.5, label='predicted % particles per bin', width=bar_width)
    ax2.set_ylabel('Counts', color='black', fontdict={'fontsize': 30, 'fontname': 'Times New Roman'})
    ax2.legend(loc='upper left', fontsize=30, bbox_to_anchor=(1.2, 1), borderaxespad=0., frameon=False)
    plt.title('Model: NN - PFR: {}kg/h - Gas Temperature: {}\u00b0C'.format(round(powder_feeders_speed_1_2_rpm, 2), Gas_temperature_C), fontdict={'fontsize': 28, 'fontname': 'Times New Roman'})

    plt.grid(True)
    ax1.tick_params(axis='both', labelsize=30)
    ax2.tick_params(axis='y', labelsize=30)

    plt.show()

except ValueError as e:
    print(e)
    print("Invalid input. Please enter a valid float value within the specified ranges.")

Enter powder feeder rate [RPM]: 2
You entered: 2.0
Enter Gas temperature [°C]: 500
Gas temperature must be between 750 and 1100°C.
Invalid input. Please enter a valid float value within the specified ranges.
